In [1]:
import pandas as pd 
import numpy as np

In [2]:
tess_data = pd.read_csv("../data/TOI_2025.10.04_08.44.51.csv", skiprows=69)
tess_data.head()

,toi,tid,tfopwg_disp,rastr,ra,decstr,dec,st_pmra,st_pmraerr1,st_pmraerr2,...,st_logg,st_loggerr1,st_loggerr2,st_logglim,st_rad,st_raderr1,st_raderr2,st_radlim,toi_created,rowupdate
0,1000.01,50365310,FP,07h29m25.85s,112.357708,-12d41m45.46s,-12.695960,-5.964,0.085,-0.085,...,4.19,0.07,-0.07,0,2.16986,0.072573,-0.072573,0,2019-07-24 15:58:33,2024-09-09 10:08:01
1,1001.01,88863718,PC,08h10m19.31s,122.580465,-05d30m49.87s,-5.513852,-4.956,0.102,-0.102,...,4.03,0.09,-0.09,0,2.01000,0.090000,-0.090000,0,2019-07-24 15:58:33,2023-04-03 14:31:04
2,1002.01,124709665,FP,06h58m54.47s,104.726966,-10d34m49.64s,-10.580455,-1.462,0.206,-0.206,...,NaN,NaN,NaN,0,5.73000,NaN,NaN,0,2019-07-24 15:58:33,2022-07-11 16:02:02
3,1003.01,106997505,FP,07h22m14.39s,110.559945,-25d12m25.26s,-25.207017,-0.939,0.041,-0.041,...,4.15,1.64,-1.64,0,NaN,NaN,NaN,0,2019-07-24 15:58:33,2022-02-23 10:10:02
4,1004.01,238597883,FP,08h08m42.77s,122.178195,-48d48m10.12s,-48.802811,-4.496,0.069,-0.069,...,4.14,0.07,-0.07,0,2.15000,0.060000,-0.060000,0,2019-07-24 15:58:33,2024-09-09 10:08:01


In [3]:
tess_data.shape

(7703, 65)

In [4]:
tess_data.columns

Index(['toi', 'tid', 'tfopwg_disp', 'rastr', 'ra', 'decstr', 'dec', 'st_pmra',
       'st_pmraerr1', 'st_pmraerr2', 'st_pmralim', 'st_pmdec', 'st_pmdecerr1',
       'st_pmdecerr2', 'st_pmdeclim', 'pl_tranmid', 'pl_tranmiderr1',
       'pl_tranmiderr2', 'pl_tranmidlim', 'pl_orbper', 'pl_orbpererr1',
       'pl_orbpererr2', 'pl_orbperlim', 'pl_trandurh', 'pl_trandurherr1',
       'pl_trandurherr2', 'pl_trandurhlim', 'pl_trandep', 'pl_trandeperr1',
       'pl_trandeperr2', 'pl_trandeplim', 'pl_rade', 'pl_radeerr1',
       'pl_radeerr2', 'pl_radelim', 'pl_insol', 'pl_insolerr1', 'pl_insolerr2',
       'pl_insollim', 'pl_eqt', 'pl_eqterr1', 'pl_eqterr2', 'pl_eqtlim',
       'st_tmag', 'st_tmagerr1', 'st_tmagerr2', 'st_tmaglim', 'st_dist',
       'st_disterr1', 'st_disterr2', 'st_distlim', 'st_teff', 'st_tefferr1',
       'st_tefferr2', 'st_tefflim', 'st_logg', 'st_loggerr1', 'st_loggerr2',
       'st_logglim', 'st_rad', 'st_raderr1', 'st_raderr2', 'st_radlim',
       'toi_created', 'rowupda

In [5]:
tess_data.drop(columns=['toi', 'tid', 'rastr', 'toi_created', 'rowupdate'], inplace=True)

In [8]:
tess_data_X = tess_data.drop(columns=['tfopwg_disp'])
tess_data_y = tess_data['tfopwg_disp']

In [11]:
tess_data_X.shape

(7703, 59)

In [13]:
tess_data_y.value_counts()

tfopwg_disp
PC     4679
FP     1197
CP      684
KP      583
APC     462
FA       98
Name: count, dtype: int64

In [19]:
tess_data_X.drop(columns=["decstr"], inplace=True)

In [20]:
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# LightGBM handles NaNs internally, so no need for manual imputation

# === Train-test split (stratified) ===
X_train, X_test, y_train, y_test = train_test_split(
    tess_data_X, tess_data_y, 
    test_size=0.2, 
    random_state=42,
    stratify=tess_data_y  # keeps class distribution balanced
)

# === Initialize LightGBM model ===
lgbm_model = lgb.LGBMClassifier(
    objective='multiclass',
    num_class=len(np.unique(tess_data_y)),
    boosting_type='gbdt',
    n_estimators=500,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

# === 5-fold Stratified Cross Validation ===
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = cross_val_score(
    lgbm_model,
    X_train,
    y_train,
    cv=cv,
    scoring='accuracy',
    n_jobs=-1
)

print(f"Average CV Accuracy: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

# === Fit on full training data ===
lgbm_model.fit(X_train, y_train)

# === Evaluate on test set ===
y_pred = lgbm_model.predict(X_test)

print("\nTest Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.094356 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9674
[LightGBM] [Info] Number of data points in the train set: 4930, number of used features: 42
[LightGBM] [Info] Start training from score -2.812735
[LightGBM] [Info] Start training from score -2.420875
[LightGBM] [Info] Start training from score -4.359960
[LightGBM] [Info] Start training from score -1.861912
[LightGBM] [Info] Start training from score -2.581516
[LightGBM] [Info] Start training from score -0.498729
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.175960 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9677
[LightGBM] [Info] Number of data points in the train set: 4929, 

Exception ignored on calling ctypes callback function <function _log_callback at 0x70f4ea9d7a60>:
Traceback (most recent call last):
  File "/home/dima/projects/nasa-challenge/.venv/lib/python3.13/site-packages/lightgbm/basic.py", line 287, in _log_callback
    def _log_callback(msg: bytes) -> None:
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, f1_score

f1_macro = make_scorer(f1_score, average='macro')
cv_scores_f1 = cross_val_score(lgbm_model, X_train, y_train, cv=cv, scoring=f1_macro)
print(f"Average CV F1 (macro): {cv_scores_f1.mean():.4f}")

In [ ]:
lgbm_model = lgb.LGBMClassifier(
    objective='multiclass',
    num_class=len(np.unique(tess_data_y)),
    class_weight='balanced',
    n_estimators=500,
    learning_rate=0.05,
    random_state=42
)
